# PyCaret Fugue Integration

[Fugue](https://github.com/fugue-project/fugue) is a low-code unified interface for different computing frameworks such as Spark, Dask and Pandas. PyCaret is using Fugue to support distributed computing scenarios.

## Hello World

### Classification

Let's start with the most standard example, the code is exactly the same as the local version, there is no magic.

In [9]:
from pycaret.datasets import get_data
from pycaret.classification import *

setup(data=get_data("juice", verbose=False), target = 'Purchase', n_jobs=1)

test_models = models().index.tolist()[:5]

,Description,Value
0,Session id,3021
1,Target,Purchase
2,Target type,Binary
3,Target mapping,"CH: 0, MM: 1"
4,Original data shape,"(1070, 19)"
5,Transformed data shape,"(1070, 19)"
6,Transformed train set shape,"(748, 19)"
7,Transformed test set shape,"(322, 19)"
8,Ordinal features,1
9,Numeric features,17


`compare_model` is also exactly the same if you don't want to use a distributed system

In [10]:
compare_models(include=test_models, n_select=2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8328,0.8994,0.7423,0.7919,0.7646,0.6355,0.6378,0.2610
dt,Decision Tree Classifier,0.7899,0.7756,0.7202,0.7199,0.7152,0.5499,0.5546,0.0770
nb,Naive Bayes,0.7618,0.8299,0.7890,0.6491,0.7098,0.5122,0.5218,0.0730
knn,K Neighbors Classifier,0.7581,0.7966,0.6004,0.7019,0.6433,0.4631,0.4689,0.0770
svm,SVM - Linear Kernel,0.5467,0.0000,0.4632,0.2017,0.2790,0.0480,0.0825,0.0540


Processing:   0%|          | 0/29 [00:00<?, ?it/s]

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3021, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        random_state=3021, splitter='best')]

Now let's make it distributed, as a toy case, on dask. The only thing changed is an additional parameter `parallel_backend`

In [11]:
from pycaret.parallel import FugueBackend

compare_models(include=test_models, n_select=2, parallel=FugueBackend("dask"))

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3736, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        random_state=7879, splitter='best')]

In order to use Spark as the execution engine, you must have access to a Spark cluster, and you must have a `SparkSession`, let's initialize a local Spark session

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Now just change `parallel_backend` to this session object, you make it run on Spark. You must understand this is a toy case. In the real situation, you need to have a SparkSession pointing to a real Spark cluster to enjoy the power of Spark

In [13]:
compare_models(include=test_models, n_select=2, parallel=FugueBackend(spark))

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=8960, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        random_state=2762, splitter='best')]

In the end, you can `pull` to get the metrics table

In [14]:
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8370,0.9077,0.7800,0.8083,0.7916,0.6580,0.6607,0.199
dt,Decision Tree Classifier,0.7901,0.7780,0.7271,0.7271,0.7250,0.5555,0.5575,0.287
knn,K Neighbors Classifier,0.7663,0.8028,0.6486,0.7308,0.6840,0.5000,0.5047,0.293
nb,Naive Bayes,0.7459,0.8226,0.7682,0.6527,0.7043,0.4850,0.4914,0.303
svm,SVM - Linear Kernel,0.5240,0.0000,0.3483,0.2100,0.2316,-0.0266,-0.0179,0.246


### Regression

It follows the same pattern as classification.

In [15]:
from pycaret.datasets import get_data
from pycaret.regression import *

setup(data=get_data("insurance", verbose=False), target = 'charges', n_jobs=1)

test_models = models().index.tolist()[:5]

,Description,Value
0,Session id,1683
1,Target,charges
2,Target type,Regression
3,Data shape,"(1338, 10)"
4,Train data shape,"(936, 10)"
5,Test data shape,"(402, 10)"
6,Ordinal features,2
7,Numeric features,3
8,Categorical features,3
9,Preprocess,True


`compare_model` is also exactly the same if you don't want to use a distributed system

In [16]:
compare_models(include=test_models, n_select=2)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,4252.7338,38419494.3379,6186.9692,0.7323,0.5671,0.4222,0.1470
lasso,Lasso Regression,4252.8304,38416936.5079,6186.7561,0.7323,0.5663,0.4222,0.0520
ridge,Ridge Regression,4266.2288,38423359.7708,6187.2227,0.7323,0.5590,0.4246,0.0510
lar,Least Angle Regression,4254.3260,38479352.3794,6191.6430,0.7318,0.5691,0.4217,0.0540
en,Elastic Net,7343.3137,91089351.7117,9525.9071,0.3729,0.7377,0.9253,0.0500


Processing:   0%|          | 0/29 [00:00<?, ?it/s]

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1,
                  normalize='deprecated', positive=False),
 Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
       normalize='deprecated', positive=False, precompute=False,
       random_state=1683, selection='cyclic', tol=0.0001, warm_start=False)]

Now let's make it distributed, as a toy case, on dask. The only thing changed is an additional parameter `parallel_backend`

In [17]:
from pycaret.parallel import FugueBackend

compare_models(include=test_models, n_select=2, parallel=FugueBackend("dask"))

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1,
                  normalize='deprecated', positive=False),
 Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
       normalize='deprecated', positive=False, random_state=897, solver='auto',
       tol=0.001)]

In order to use Spark as the execution engine, you must have access to a Spark cluster, and you must have a `SparkSession`, let's initialize a local Spark session

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Now just change `parallel_backend` to this session object, you make it run on Spark. You must understand this is a toy case. In the real situation, you need to have a SparkSession pointing to a real Spark cluster to enjoy the power of Spark

In [19]:
compare_models(include=test_models, n_select=2, parallel=FugueBackend(spark))

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1,
                  normalize='deprecated', positive=False),
 Lars(copy_X=True, eps=2.220446049250313e-16, fit_intercept=True, fit_path=True,
      jitter=None, n_nonzero_coefs=500, normalize='deprecated',
      precompute='auto', random_state=4780, verbose=False)]

In the end, you can `pull` to get the metrics table

In [20]:
pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,4194.8289,3.787007e+07,6127.6806,0.7422,0.5678,0.4248,0.315
lar,Least Angle Regression,4201.5526,3.726411e+07,6045.4291,0.7372,0.5832,0.4275,0.272
lasso,Lasso Regression,4256.6163,3.724654e+07,6078.0660,0.7360,0.6049,0.4448,0.254
ridge,Ridge Regression,4282.2894,3.838556e+07,6164.1926,0.7101,0.6324,0.4472,0.326
en,Elastic Net,7228.6526,9.031280e+07,9427.7589,0.3619,0.7370,0.9231,0.316


As you see, the results from the distributed versions can be different from your local versions. In the later sections, we will show how to make them identical.

### Time Series

It follows the same pattern as classification.


In [9]:
from pycaret.datasets import get_data
from pycaret.time_series import *

exp = TSForecastingExperiment()
exp.setup(data=get_data('airline', verbose=False), fh=12, fold=3, fig_kwargs={'renderer': 'notebook'}, session_id=42)

test_models = exp.models().index.tolist()[:5]

,Description,Value
0,session_id,42
1,Target,Number of airline passengers
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(144, 1)"
5,Transformed data shape,"(144, 1)"
6,Transformed train set shape,"(132, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [10]:
best_baseline_models = exp.compare_models(include=test_models, n_select=3)
best_baseline_models

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
arima,ARIMA,0.6830,0.6735,20.0069,22.2199,0.0501,0.0507,0.8677,0.1400
snaive,Seasonal Naive Forecaster,1.1479,1.0945,33.3611,35.9139,0.0832,0.0879,0.6072,0.0200
polytrend,Polynomial Trend Forecaster,1.6523,1.9202,48.6301,63.4299,0.1170,0.1216,-0.0784,0.0167
naive,Naive Forecaster,2.3599,2.7612,69.0278,91.0322,0.1569,0.1792,-1.2216,1.0333
grand_means,Grand Means Forecaster,5.5306,5.2596,162.4117,173.6492,0.4000,0.5075,-7.0462,0.6667


Processing:   0%|          | 0/29 [00:00<?, ?it/s]

[ARIMA(maxiter=50, method='lbfgs', order=(1, 0, 0), out_of_sample_size=0,
       scoring='mse', scoring_args=None, seasonal_order=(0, 1, 0, 12),
       start_params=None, suppress_warnings=False, trend=None,
       with_intercept=True),
 NaiveForecaster(sp=12, strategy='last', window_length=None),
 PolynomialTrendForecaster(degree=1, regressor=None, with_intercept=True)]

In [11]:
from pycaret.parallel import FugueBackend

best_baseline_models = exp.compare_models(include=test_models, n_select=3, parallel=FugueBackend("dask"))
best_baseline_models

[NaiveForecaster(sp=1, strategy='mean', window_length=None),
 NaiveForecaster(sp=1, strategy='last', window_length=None),
 PolynomialTrendForecaster(degree=1, regressor=None, with_intercept=True)]

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [15]:
from pycaret.parallel import FugueBackend

best_baseline_models = exp.compare_models(include=test_models[:2], n_select=3, parallel=FugueBackend(spark))
best_baseline_models

[NaiveForecaster(sp=1, strategy='mean', window_length=None),
 NaiveForecaster(sp=1, strategy='last', window_length=None)]

In [17]:
exp.pull()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
grand_means,Grand Means Forecaster,5.5306,5.2596,162.4117,173.6492,0.4,0.5075,-7.0462,1.8200
naive,Naive Forecaster,2.3599,2.7612,69.0278,91.0322,0.1569,0.1792,-1.2216,1.8133


## A more practical case

The above examples are pure toys, to make things work perfectly in a distributed system you must be careful about a few things

### Use a lambda instead of a dataframe in setup

If you directly provide a dataframe in `setup`, this dataset will need to be sent to all worker nodes. If the dataframe is 1G, you have 100 workers, then it is possible your dirver machine will need to send out up to 100G data (depending on specific framework's implementation), then this data transfer becomes a bottleneck itself. Instead, if you provide a lambda function, it doesn't change the local compute scenario, but the driver will only send the function reference to workers, and each worker will be responsible to load the data by themselves, so there is no heavy traffic on the driver side.

### Be deterministic

You should always use `session_id` to make the distributed compute deterministic, otherwise, for the exactly same logic you could get drastically different selection for each run.

### Set n_jobs

It is important to be explicit on n_jobs when you want to run something distributedly, so it will not overuse the local/remote resources. This can also avoid resrouce contention, and make the compute faster.

In [1]:
from pycaret.datasets import get_data
from pycaret.classification import *

setup(data_func=lambda: get_data("juice", verbose=False, profile=False), target = 'Purchase', session_id=0, n_jobs=1);

,Description,Value
0,Session id,0
1,Target,Purchase
2,Target type,Binary
3,Target mapping,"CH: 0, MM: 1"
4,Original data shape,"(1070, 19)"
5,Transformed data shape,"(1070, 19)"
6,Transformed train set shape,"(748, 19)"
7,Transformed test set shape,"(322, 19)"
8,Ordinal features,1
9,Numeric features,17


### Set the appropriate batch_size

`batch_size` parameter helps adjust between load balence and overhead. For each batch, setup will be called only once. So

| Choice |Load Balance|Overhead|Best Scenario|
|---|---|---|---|
|Smaller batch size|Better|Worse|`training time >> data loading time` or `models ~= workers`|
|Larger batch size|Worse|Better|`training time << data loading time` or `models >> workers`|

The default value is set to `1`, meaning we want the best load balance.

### Display progress

In development, you can enable visual effect by `display_remote=True`, but meanwhile you must also enable [Fugue Callback](https://fugue-tutorials.readthedocs.io/tutorials/advanced/rpc.html) so that the driver can monitor worker progress. But it is recommended to turn off display in production.

In [2]:
from pycaret.parallel import FugueBackend

fconf = {
    "fugue.rpc.server": "fugue.rpc.flask.FlaskRPCServer",  # keep this value
    "fugue.rpc.flask_server.host": "0.0.0.0",  # the driver ip address workers can access
    "fugue.rpc.flask_server.port": "3333",  # the open port on the dirver
    "fugue.rpc.flask_server.timeout": "2 sec",  # the timeout for worker to talk to driver
}

be = FugueBackend("dask", fconf, display_remote=True, batch_size=3, top_only=False)
compare_models(n_select=2, parallel=be)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8302,0.8953,0.7656,0.7863,0.7721,0.6373,0.6412,0.118
ridge,Ridge Classifier,0.8275,0.0000,0.7515,0.7926,0.7659,0.6302,0.6360,0.139
lr,Logistic Regression,0.8195,0.8948,0.7123,0.8004,0.7467,0.6080,0.6163,0.311
gbc,Gradient Boosting Classifier,0.8128,0.8877,0.7299,0.7700,0.7463,0.5985,0.6019,0.207
catboost,CatBoost Classifier,0.8127,0.8962,0.7336,0.7729,0.7464,0.5989,0.6054,1.137
ada,Ada Boost Classifier,0.8048,0.8702,0.6979,0.7721,0.7280,0.5770,0.5830,0.166
rf,Random Forest Classifier,0.8047,0.8792,0.7156,0.7626,0.7348,0.5808,0.5849,0.236
xgboost,Extreme Gradient Boosting,0.7954,0.8781,0.7154,0.7416,0.7261,0.5632,0.5655,0.359
lightgbm,Light Gradient Boosting Machine,0.7941,0.8780,0.7155,0.7397,0.7243,0.5605,0.5636,0.138
et,Extra Trees Classifier,0.7752,0.8556,0.6803,0.7224,0.6948,0.5182,0.5239,0.207


Processing:   0%|          | 0/16 [00:00<?, ?it/s]

[LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=None, solver='svd',
                            store_covariance=False, tol=0.0001),
 RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                 max_iter=None, normalize='deprecated', positive=False,
                 random_state=0, solver='auto', tol=0.001)]

22/05/30 06:50:37 WARN Utils: Your hostname, codespaces_98dd70 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/05/30 06:50:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/30 06:50:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/codespace/.local/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1545: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/home/codespace/.local/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1545: FutureWarning: In the percentage error metric

### Custom Metrics

You can add custom metrics like before. But in order to make the scorer distributable, it must be serializable. A common function should be fine, but if inside the function, it is using some global variables that are not serializable (for example an `RLock` object), it can cause issues. So try to make the custom function independent from global variables.

In [3]:
def score_dummy(y_true, y_prob, axis=0):
    return 0.0

add_metric(id = 'mydummy',
               name = 'DUMMY',
               score_func = score_dummy,
               target = 'pred_proba',
               greater_is_better = True,
              )

Name                                                             DUMMY
Display Name                                                     DUMMY
Score Function                <function score_dummy at 0x7f473f7e8b80>
Scorer               make_scorer(score_dummy, needs_proba=True, err...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                 True
Multiclass                                                        True
Custom                                                            True
Name: mydummy, dtype: object

Adding a function in a class instance is also ok, but make sure all member variables in the class are serializable.

In [4]:
test_models = models().index.tolist()[:5]
compare_models(include=test_models, n_select=2, sort="DUMMY", parallel=FugueBackend("dask"))

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=1, n_neighbors=5, p=2,
                      weights='uniform')]

In [5]:
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,DUMMY,TT (Sec)
lr,Logistic Regression,0.8195,0.8948,0.7123,0.8004,0.7467,0.6080,0.6163,0.0,0.263
knn,K Neighbors Classifier,0.7661,0.8103,0.6553,0.7102,0.6785,0.4956,0.4992,0.0,0.082
svm,SVM - Linear Kernel,0.5238,0.0000,0.5464,0.2413,0.3343,0.0462,0.0817,0.0,0.090
nb,Naive Bayes,0.7553,0.8340,0.7475,0.6550,0.6954,0.4929,0.4989,0.0,0.129
dt,Decision Tree Classifier,0.7606,0.7431,0.6724,0.6873,0.6775,0.4875,0.4899,0.0,0.075


In [6]:
class Scores:
    def score_dummy2(self, y_true, y_prob, axis=0):
        return 1.0
    
scores = Scores()

add_metric(id = 'mydummy2',
               name = 'DUMMY2',
               score_func = scores.score_dummy2,
               target = 'pred_proba',
               greater_is_better = True,
              )

Name                                                            DUMMY2
Display Name                                                    DUMMY2
Score Function       <bound method Scores.score_dummy2 of <__main__...
Scorer               make_scorer(score_dummy2, needs_proba=True, er...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                 True
Multiclass                                                        True
Custom                                                            True
Name: mydummy2, dtype: object

In [7]:
compare_models(include=test_models, n_select=2, sort="DUMMY2", parallel=FugueBackend("dask"))

[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=1, n_neighbors=5, p=2,
                      weights='uniform'),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        random_state=0, splitter='best')]

In [8]:
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,DUMMY,DUMMY2,TT (Sec)
knn,K Neighbors Classifier,0.7661,0.8103,0.6553,0.7102,0.6785,0.4956,0.4992,0.0,1.0,0.111
dt,Decision Tree Classifier,0.7606,0.7431,0.6724,0.6873,0.6775,0.4875,0.4899,0.0,1.0,0.109
lr,Logistic Regression,0.8195,0.8948,0.7123,0.8004,0.7467,0.6080,0.6163,0.0,1.0,0.279
nb,Naive Bayes,0.7553,0.8340,0.7475,0.6550,0.6954,0.4929,0.4989,0.0,1.0,0.075
svm,SVM - Linear Kernel,0.5238,0.0000,0.5464,0.2413,0.3343,0.0462,0.0817,0.0,0.0,0.055


## Notes

### Spark settings

It is highly recommended to have only 1 worker on each Spark executor, so the worker can fully utilize all cpus (set `spark.task.cpus`). Also when you do this you should explicitly set `n_jobs` in `setup` to the number of cpus of each executor.

```python
executor_cores = 4

spark = SparkSession.builder.config("spark.task.cpus", executor_cores).config("spark.executor.cores", executor_cores).getOrCreate()

setup(data=get_data("juice", verbose=False, profile=False), target = 'Purchase', session_id=0, n_jobs=executor_cores)

compare_models(n_select=2, parallel=FugueBackend(spark))
```

### Databricks

On Databricks, `spark` is the magic variable representing a SparkSession. But there is no difference to use. You do the exactly same thing as before:

```python
compare_models(parallel=FugueBackend(spark))
```

But Databricks, the visualization is difficult, so it may be a good idea to do two things:

* Set `verbose` to False in `setup`
* Set `display_remote` to False in `FugueBackend`

### Dask

Dask has fake distributed modes such as the default (multi-thread) and multi-process modes. The default mode will just work fine (but they are actually running sequentially), and multi-process doesn't work for PyCaret for now because it messes up with PyCaret's global variables. On the other hand, any Spark execution mode will just work fine.

### Local Parallelization

For practical use where you try non-trivial data and models, local parallelization (The eaiest way is to use local Dask as backend as shown above) normally doesn't have performance advantage. Because it's very easy to overload the CPUS on training, increasing the contention of resources. The value of local parallelization is to verify the code and give you confidence that the distributed environment will provide the expected result with much shorter time.

### How to develop 

Distributed systems are powerful but you must follow some good practices to use them:

1. **From small to large:** initially, you must start with a small set of data, for example in `compare_model` limit the models you want to try to a small number of cheap models, and when you verify they work, you can change to a larger model collection.
2. **From local to distributed:** you should follow this sequence: verify small data locally then verify small data distributedly and then verify large data distributedly. The current design makes the transition seamless. You can do these sequentially: `parallel=None` -> `parallel=FugueBackend()` -> `parallel=FugueBackend(spark)`. In the second step, you can replace with a local SparkSession or local dask.